In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
BATCH_SIZE=512 # 批次大小
EPOCHS=5 # 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多
 

In [3]:
DOWNLOAD_MNIST = False

train_data = datasets.MNIST('../MNIST_data/', train=True, download=DOWNLOAD_MNIST,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

test_data = datasets.MNIST('../MNIST_data/', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=BATCH_SIZE, shuffle=True)

print("train_data:", train_data.data.size()) #torch.size() 查看tensor的维度
print("test_data:", test_data.data.size())

train_data: torch.Size([60000, 28, 28])
test_data: torch.Size([10000, 28, 28])


In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1 ,10,5) # 24x24 input_channel output_channel kernel_size strides=1 (28-5+1) / 1 = 24
        self.pool = nn.MaxPool2d(2,2) # 12x12
        self.conv2=nn.Conv2d(10,20,3) # 10x10 (12-3+1)/1 = 10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0) #512,BATCH_SIZE
        #print(in_size)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = self.pool(out)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)# the size -1 is inferred from other dimensions, result:(BATCH_SIZE , N) 证明输入网络的数据以batch为单位
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out


In [5]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()# train 模式 启用 BatchNormalization 和 Dropout
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)#把tensor送到相应设备上
        optimizer.zero_grad() #梯度清零
        output = model(data) #前向推理，自动调用model.forward 
        loss = F.nll_loss(output, target) #计算NLLLoss 负对数似然损失 根据标签把对应位置的值取出来去掉符号相加做平均 而交叉熵 = Softmax–Log–NLLLoss合并成一步
        loss.backward()#有了loss之后反向传播
        optimizer.step()#更新参数
        if(batch_idx+1) % 30 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())) #用item得到元素值
                
def test(model, device, test_loader):
    model.eval()# eval 模式 关闭 BatchNormalization 和 Dropout
    test_loss = 0
    correct = 0
    with torch.no_grad(): #关闭梯度计算
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))





In [6]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters()) #使用adam优化器
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.347907
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.187541
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.161312

Test set: Average loss: 0.0902, Accuracy: 9746/10000 (97%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.156242
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.070588
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.077610

Test set: Average loss: 0.0632, Accuracy: 9806/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.075663
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.050295
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.034362

Test set: Average loss: 0.0416, Accuracy: 9863/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.031840
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.058905
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.028396

Test set: Average loss: 0.0382, Accuracy: 9872/10000 (99%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.037170
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.049495
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.024958

Test 

In [14]:
print(model)

ConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2000, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [16]:
MODEL_PATH = './models/convnet_mnist.pth'
torch.save(model.state_dict(),MODEL_PATH) #只存字典，直接输入model的话会保存整个模型 

In [24]:
#注意这里必须送到DEVICE中去 原因是cpu和gpu上的tensor不一样
#否则报错Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same
model_load = ConvNet().to(DEVICE) 
model_load.load_state_dict(torch.load(MODEL_PATH))
model.eval()
# model.train() ：启用 BatchNormalization 和 Dropout
# model.eval() ：不启用 BatchNormalization 和 Dropout

ConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2000, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [25]:
test(model_load, DEVICE, test_loader)


Test set: Average loss: 0.0373, Accuracy: 9913/10000 (99%)

